In [2]:
pip install --upgrade --user ortools

     |████████████████████████████████| 27.9MB 170kB/s 
     |████████████████████████████████| 1.3MB 42.6MB/s 


In [2]:
from __future__ import print_function
import sys
from math import sin, cos, sqrt, atan2, radians
from array import *
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import folium
import numpy as np
import pandas as pd
from collections import namedtuple
Radius = 6373.0 #of earth
################################################ PARAMETERS ################################################################################
#LOCATIONS (in deg)
way_pt=[(48.154106, 11.58232), # location 0 - Leopoldstrasse 5, Munchen Department of Infectious and Tropical Medicine
(48.18684, 11.52782),    # location 1
(48.20076, 11.66443),    # location 2
(48.17703, 11.49623),     # location 3
(48.17477, 11.51842),   # location 4
(48.17656, 11.60135),  # location 5
(48.18484, 11.65459),  # location 6
(48.16805, 11.5563),  # location 7
(48.16722, 11.62712),  # location 8
(48.14821, 11.60172),  # location 9
(48.15624, 11.67071),  # location 10
(48.13997, 11.50305),  # location 11
(48.13913, 11.5182),  # location 12
(48.12883, 11.51953),  # location 13
(48.12803, 11.62192),  # location 14
(48.13, 11.48954),    # location 15
(48.11729, 11.65595)]  # location 16

drones_num =4 #number is drones in service

output_nodes = [] #list to combine all the routes

# using Munich city center coordinates , 
center_lat = 48.158757
center_lon = 11.565857

p1 = (48.154106, 11.58232)

nodes = []
out_map = folium.Map(location=[center_lat, center_lon], zoom_start=12.25)
for i in range(len(way_pt)):
  nodes.append(folium.CircleMarker(way_pt[i], color='black', fill_color='black', radius=12))
for i in nodes:
    i.add_to(out_map)
folium.Marker(location=p1, icon=folium.Icon(color='green')).add_to(out_map)
out_map.save('initial_out.html') #save the folium map as html file
out_map #display the map

#angle of the arrow
def get_bearing(p1, p2):    
    p1_lat, p1_lon = p1
    p2_lat, p2_lon = p2

    long_diff = np.radians(p2_lon - p1_lon)
    
    lat1 = np.radians(p1_lat)
    lat2 = np.radians(p2_lat)
    
    x = np.sin(long_diff) * np.cos(lat2)
    y = (np.cos(lat1) * np.sin(lat2) 
        - (np.sin(lat1) * np.cos(lat2) 
        * np.cos(long_diff)))
    bearing = np.degrees(np.arctan2(x, y))
    
    # adjusting for compass bearing
    if bearing < 0:
        return bearing + 360
    return bearing

#getting the arrow points
def get_arrows_mod(locations, color, size=12, n_arrows=1):
    p1 = locations[0]
    p2 = locations[len(locations)-1]
    
    rotation = get_bearing(p1, p2) - 90

    arrow_lats = []
    arrow_lons = []
    arrow_lats_inter = []
    arrow_lons_inter = []
    arrows_inter = []
    for i in range(len(locations)-1):
      lat, lon = locations[i]
      lat_e, lon_e = locations[i+1]
      if i+1 != len(locations):
        #folium.Marker(locations[i+1], icon=folium.Icon(color='blue')).add_to(some_map)
        arrow_lats.append(lat)
        arrow_lons.append(lon)
      arrow_lats_inter = (np.linspace(lat, lat_e, n_arrows + 2)[1:n_arrows+1])
      arrow_lons_inter = (np.linspace(lon, lon_e, n_arrows + 2)[1:n_arrows+1])
      #creating each "arrow" and appending them to our arrows list
      for points in zip(arrow_lats_inter, arrow_lons_inter):
          arrows_inter.append(folium.RegularPolygonMarker(location=points, color=color,
                        fill_color=color, number_of_sides=3, 
                        radius=6, rotation=get_bearing(locations[i], locations[i+1]) - 90))
    arrows = []
    #print(arrow_lats, arrow_lons)
    #creating each "arrow" and appending them to our arrows list
    for points in zip(arrow_lats, arrow_lons):
        arrows.append(folium.CircleMarker(location=points, color='black', fill_color=color, radius=size, rotation=rotation))
     
    #print(arrow_lats_inter)
    return arrows,arrows_inter

#calculating the route
def geo_distance(la1,lo1,la2,lo2):
	lat1 = radians(la1)
	lon1 = radians(lo1)
	lat2 = radians(la2)
	lon2 = radians(lo2)
	dlon = lon2 - lon1
	dlat = lat2 - lat1
	a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
	c = 2 * atan2(sqrt(a), sqrt(1 - a))
	distance = Radius * c
	return distance
# print (geo_distance (way_pt[0][0],way_pt[0][1],way_pt[1][0],way_pt[1][1]))

def create_data_model():
	dist_matrix = [[0 for i in range( len(way_pt) )] for j in range( len(way_pt) )]
	for i in range(len(way_pt)):
		for j in range(len(way_pt)):
			dist_matrix[i][j] = int(1000*geo_distance(way_pt[i][0], way_pt[i][1], way_pt[j][0], way_pt[j][1]))
	data = {}
	data['distance_matrix']=dist_matrix
	data['num_vehicles']=drones_num
	data['depot']=0
	return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        out = []
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            out.append(way_pt[manager.IndexToNode(index)])
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        out.append(p1)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
        output_nodes.append(out)
    print('Maximum of the route distances: {}m'.format(max_route_distance))


def main():
	
	"""Solve the CVRP problem."""
	# Instantiate the data problem.

	data = create_data_model()

	# Create the routing index manager.
	manager = pywrapcp.RoutingIndexManager (len(data['distance_matrix']), data['num_vehicles'], data['depot'])

	# Create Routing Model.
	routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
	def distance_callback(from_index, to_index):
		"""Returns the distance between the two nodes."""
		# Convert from routing variable Index to distance matrix NodeIndex.
		from_node = manager.IndexToNode(from_index)
		to_node = manager.IndexToNode(to_index)
		return data['distance_matrix'][from_node][to_node]

	transit_callback_index = routing.RegisterTransitCallback(distance_callback)

	# Define cost of each arc.
	routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

	# Add Distance constraint.
	dimension_name = 'Distance'
	routing.AddDimension(
		transit_callback_index,
		0,  # no slack
		50000,  # vehicle maximum travel distance
		True,  # start cumul to zero
		dimension_name)
	distance_dimension = routing.GetDimensionOrDie(dimension_name)
	distance_dimension.SetGlobalSpanCostCoefficient(100)

	# Setting first solution heuristic.
	search_parameters = pywrapcp.DefaultRoutingSearchParameters()
	search_parameters.first_solution_strategy = (
		routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
	solution = routing.SolveWithParameters(search_parameters)

	# Print solution on console.
	if solution:
		print_solution(data, manager, routing, solution)
  
if __name__=='__main__':
	main()
 
color = ['blue', 'red', 'green', 'orange']
for j in range(drones_num):
  drone = output_nodes[j]
  for i in range(len(drone)-1):
    folium.PolyLine(locations=[drone[i], drone[i+1]], color=color[j]).add_to(out_map)
  arrow, arrow_inter = get_arrows_mod(drone,color=color[j], n_arrows=1)
  for a in arrow:
    a.add_to(out_map)
  for a in arrow_inter:
    a.add_to(out_map)

folium.Marker(location=p1, icon=folium.Icon(color='green')).add_to(out_map)
out_map.save('final_out.html')
out_map


Route for vehicle 0:
 0 ->  10 ->  16 ->  14 -> 0
Distance of the route: 17954m

Route for vehicle 1:
 0 ->  9 ->  8 ->  6 ->  2 ->  5 -> 0
Distance of the route: 17419m

Route for vehicle 2:
 0 ->  12 ->  11 ->  15 ->  13 -> 0
Distance of the route: 15336m

Route for vehicle 3:
 0 ->  4 ->  3 ->  1 ->  7 -> 0
Distance of the route: 14963m

Maximum of the route distances: 17954m
